<a href="https://colab.research.google.com/github/RajaPaul92/Data-Science/blob/master/GGAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [ ]:
# Step 1: Read the csv file into a dataframe
# Dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress
input_df = pd.read_csv('/sample_data_set_2.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='sample_data_sets_2',
            db_name='default.db')
 
# Step 3: Write the SQL query in a string variable
sql_query_string = """
    SELECT App, Ad_Group, State, sum(Revenue) as Revenue,
    sum(Ad_Requests) as Ad_Requests, sum(First_Clicks) as First_Clicks,
    sum(Second_Clicks) as Second_Clicks 
    from sample_data_sets_2
    where State != 'Unresolved'
    group by App, Ad_Group, State
    order by Ad_Requests desc
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

2021-11-23 06:27:30 INFO: SQL DB default.db created
2021-11-23 06:27:30 INFO: SQL Table sample_data_sets_2 created with 12 columns
2021-11-23 06:27:30 INFO: 48293 rows uploaded to sample_data_sets_2


,App,Ad_Group,State,Revenue,Ad_Requests,First_Clicks,Second_Clicks
0,Callbreak Multiplayer,non stop fun,Uttar Pradesh,0.00,49836654,1,0
1,Callbreak Multiplayer,non stop fun,Bihar,0.00,30514913,0,0
2,Callbreak Multiplayer,non stop fun,Rajasthan,0.00,29867450,0,0
3,Callbreak Multiplayer,Callbreak Multiplayer2_Inmobi_No Floor,Uttar Pradesh,18.99,24921913,4573,1496
4,Callbreak Multiplayer,Danish&satish,Uttar Pradesh,0.00,24921913,0,0
...,...,...,...,...,...,...,...
10681,Shortpedia,SDK X Test Ads,Delhi,0.00,0,0,0
10682,Tips for ZOOM Cloud Meetings Video Conferences,SDK X Test Ads 2,Chhattisgarh,0.00,0,0,0
10683,Tips for ZOOM Cloud Meetings Video Conferences,SDK X Test Ads 2,Kerala,0.00,0,0,0
10684,VR Thrills,None,West Bengal,0.00,0,0,0


In [ ]:
sql_query_string = """
    SELECT count(distinct(App)) as App, count(distinct(Ad_Group)) as Ad_Group, count(distinct(State)) as State, sum(Revenue) as Revenue,
    sum(Ad_Requests) as Ad_Requests, sum(Ad_Responses) as Ad_Responses,sum(Impressions) as Impressions, sum(First_Clicks) as First_Clicks,
    sum(Second_Clicks) as Second_Clicks 
    from sample_data_sets_2
    where State != 'Unresolved' 
    order by Ad_Requests desc
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,App,Ad_Group,State,Revenue,Ad_Requests,Ad_Responses,Impressions,First_Clicks,Second_Clicks
0,174,180,35,2734.83,3225360854,208759368,186717394,4711944,1252553


In [ ]:
sql_query_string = """
    SELECT App, Ad_Group, State, sum(Revenue) as Revenue,
    sum(Ad_Requests) as Ad_Requests, sum(Ad_Responses) as Ad_Responses,sum(First_Clicks) as First_Clicks,
    sum(Second_Clicks) as Second_Clicks 
    from sample_data_sets_2
    where State != 'Unresolved'
    group by App, Ad_Group, State
    order by Ad_Responses desc
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df
result_df.to_csv('output1.csv')

In [ ]:
sql_query_string = """
    SELECT App, count(distinct(Ad_Group)) as Ad_Group, count(distinct(State)) as State , sum(Revenue) as Revenue,
    sum(Ad_Requests) as Ad_Requests, sum(Ad_Responses) as Ad_Responses,sum(Impressions) as Impressions,sum(First_Clicks) as First_Clicks,
    sum(Second_Clicks) as Second_Clicks 
    from sample_data_sets_2
    where State != 'Unresolved'
    group by App 
    order by Revenue desc
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df
result_df.to_csv('output2.csv')

In [ ]:
sql_query_string = """
    SELECT Ad_Group, count(distinct(App)) as App, count(distinct(State)) as State , sum(Revenue) as Revenue,
    sum(Ad_Requests) as Ad_Requests, sum(Ad_Responses) as Ad_Responses,sum(Impressions) as Impressions,sum(First_Clicks) as First_Clicks,
    sum(Second_Clicks) as Second_Clicks 
    from sample_data_sets_2
    where State != 'Unresolved'
    group by Ad_Group 
    order by Revenue desc
"""

result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df
# result_df.to_csv('output3.csv')

,Ad_Group,App,State,Revenue,Ad_Requests,Ad_Responses,Impressions,First_Clicks,Second_Clicks
0,MY11Circle_IN_SG_1500,11,35,271.80,146603365,21525735,18617506,518639,133782
1,IPL banner,11,35,260.04,144661626,21051409,18333736,469011,122081
2,Callbreak Multiplayer2_Inmobi_No Floor,1,35,256.23,122961466,3382838,2699239,80714,26020
3,Dream11_NonCore,4,35,234.24,146327425,54321145,50016385,487935,120377
4,Traffic bangalore_Admob_New Banner,1,31,207.41,890513,850610,644277,8350,0
...,...,...,...,...,...,...,...,...,...
176,revshare,84,34,0.00,68694060,4527360,4201832,156270,155438
177,sling kong_Admob_Android_mid,1,3,0.00,28,7,1,0,0
178,sling kong_admob_ios_high,1,4,0.00,118,65,0,0,0
179,sling kong_admob_ios_mid,1,3,0.00,112,32,4,1,0
